In [21]:
import torch
from torch.nn import Linear, BCEWithLogitsLoss
from torch.optim import Adam
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import to_networkx
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
#from torch_geometric.data import DataLoader
from torch_geometric.loader import DataLoader
import networkx as nx
import matplotlib.pyplot as plt

In [5]:
dataset = TUDataset(root='../data/', name='MUTAG')

In [13]:
embedding_size = 32
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        
        #GCN Layers
        self.input = GCNConv(dataset.num_features, embedding_size)
        self.conv1 = GCNConv(embedding_size, embedding_size)
        self.conv2 = GCNConv(embedding_size, embedding_size)
        self.conv3 = GCNConv(embedding_size, embedding_size)
        self.output = Linear(embedding_size*2, dataset.num_classes)
        
    def forward(self, x, edge_index, batch_index):
        out = self.input(x, edge_index)
        out = F.relu(out)
        out = self.conv1(out, edge_index)
        out = F.relu(out)
        out = self.conv2(out, edge_index)
        out = F.relu(out)
        out = self.conv3(out, edge_index)
        out = F.relu(out)
        
        global_pool = torch.cat([gmp(out, batch_index), gap(out, batch_index)], dim=1)
        
        out = self.output(global_pool)
        
        return out, global_pool

In [15]:
model = GCN()
print(model)

GCN(
  (input): GCNConv(7, 32)
  (conv1): GCNConv(32, 32)
  (conv2): GCNConv(32, 32)
  (conv3): GCNConv(32, 32)
  (output): Linear(in_features=64, out_features=2, bias=True)
)


In [23]:
loss_fn = BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

GRAPH_BATCH_SIZE = 8
train_loader = DataLoader(dataset[:int(len(dataset)*0.8)], batch_size=GRAPH_BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset[int(len(dataset)*0.8):], batch_size=GRAPH_BATCH_SIZE, shuffle=True)

In [55]:
def train_model(data):
    for batch in data:
        batch.to(device)
        optimizer.zero_grad()
        prediction, embedding = model(batch.x.float(), batch.edge_index, batch.batch)
        print(prediction)
        loss = loss_fn(prediction, batch.y)
        loss.backward()
        optimizer.step()
    return loss, embedding

In [56]:
losses = []
for epoch in range(20):
    loss, embedding = train_model(train_loader)
    losses.append(loss)
    if epoch % 5 == 0:
        print(f"Epoch {epoch} | Train Loss: {loss:.3f}")

tensor([[ 0.0704, -0.0046],
        [ 0.0602, -0.0108],
        [ 0.0588, -0.0100],
        [ 0.0559, -0.0094],
        [ 0.0554, -0.0106],
        [ 0.0566, -0.0093],
        [ 0.0548, -0.0116],
        [ 0.0571, -0.0101]], grad_fn=<AddmmBackward0>)


ValueError: Target size (torch.Size([8])) must be the same as input size (torch.Size([8, 2]))